# Problem 9.1: Simple Variational Autoencoder (VAE)

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Robert Bamler at University of Tuebingen)

- This notebook is part of Problem Set 9, published on 6 June 2022 and discussed on 8 June 2022.
- You can download the full problem set (and solutions) from the [course website](https://robamler.github.io/teaching/compress22/).

This problem guides you through the implementation of a simple variational autoencoder (VAE) for hand-written digits (MNIST).
On the next problem set, we will modify the VAE that you implement here so that it can be used for *lossy* data compression.

## Preparation

Let's load some libraries, set some global configurations, and load the training and test sets.

In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
USE_CUDA = False # change to True if you have a compatible GPU
RANDOM_SEED = 20220625
BATCH_SIZE = 100
EPOCHS = 50
BATCHES_BETWEEN_LOGGING = 100
LATENT_DIM = 8

In [ ]:
torch.manual_seed(RANDOM_SEED)
device = torch.device("cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

## Visualize some of the training data

The training data consists of black-and-white handwritten digits with a resolution of 28x28 pixels.

In [ ]:
for batch, _ in train_loader:
    ax = plt.imshow(make_grid(batch[:64]).permute((1, 2, 0)))
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    break

## Encoder and Decoder Model

The classes `Encoder` and `Decoder` define two neural networks
- `Encoder` takes a batch of images $x$ as input and outputs the parameters of the variational distribution $Q_\phi(Z\,|\,X\!=\!x) = \mathcal N\big(\mu_\phi(x),\text{diag}(\sigma^2_\phi(x))\big)$.
  More precisely, $\phi$ are the weights of the neural network, and its output is a tuple $(\mu_\phi(x), \log \sigma^2_\phi(x))$.
  Here, we output the log-variances rather than the variances so that we can ensure that the variances are always positive.
- `Decoder` takes a latent representation $z$ as input and outputs the parameters $\hat x_\theta(z)$ of the likelihood $P_\theta(X\,|\,Z\!=\!z)$.
  Here, $\theta$ are the weights of the neural network, and we model the likelihood $P_\theta(X\,|\,Z\!=\!z)$ as a Bernoulli distribution, which is a fancy way of saying that each pixel $X_i$ can be either black or white, and it is white with probability $\hat x_\theta(z) \in(0,1)$.

### Your Task

Nothing to implement for you yet.
Read the above explanation and the model definitions below and make sure you understand what's going on.

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=5)
        self.fc_mean = nn.Linear(in_features=8*8*8, out_features=LATENT_DIM)
        self.fc_log_variance = nn.Linear(in_features=8*8*8, out_features=LATENT_DIM)

    def forward(self, x):
        h = torch.relu(self.conv1(x))
        h = torch.relu(self.conv2(h)).view(-1, 8 * 8 * 8)
        q_mean = self.fc_mean(h)  # (no activation function on last layer)
        q_log_variance = self.fc_log_variance(h)  # (no activation function on last layer)
        return q_mean, q_log_variance

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(in_features=LATENT_DIM, out_features=8*8*8)
        self.deconv1 = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=5)
        self.deconv2 = nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=5, stride=2, output_padding=1)

    def forward(self, z):
        h = torch.relu(self.fc(z)).view(-1, 8, 8, 8)
        h = torch.relu(self.deconv1(h))
        return torch.sigmoid(self.deconv2(h))

## Reparameterization Gradients

In order to maximize the ELBO we will use reparameterization gradients as introduced in Problem 8.2 (b) on Problem Set 8.


### Your Task

Fill in the gaps labeled `TODO` below.
Note that the second function argument is $\texttt{log\_variance} = \log(\sigma^2)$ where $\sigma$ is the (vector of) standard deviations of $Q$.

In [ ]:
def reparameterize(mean, log_variance):
    std = torch.exp(TODO) # calculate the standard deviations of Q
    eps = torch.randn_like(std) # draws standard normal distributed random numbers
    return TODO # see Eq. 7 on Problem Set 8

## Analytic Calculation of KL-Divergence

On Problem Set 8, we derived several equivalent formulations of the evidence lower bound (ELBO).
In this problem, we'll use the formulation given in Eq. 4 of Problem Set 8, i.e.,

$$
\text{ELBO}(\phi) = \mathbb E_{Q_\phi(Z|\mathbf{X}=\mathbf{x})}\big[ \log P(\mathbf{X}\! =\!\mathbf{x}\,|\,Z)\big] - D_\text{KL}\big(Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z)\big).
$$

Here, we *estimate* the first term on the right-hand side by drawing a random sample from $z\sim Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x})$ and then evaluating $ \log P(\mathbf{X}\! =\!\mathbf{x}\,|\,Z\!=\!z)$ for this sample.
By contrast, the second term on the right-hand side can be evaluated analytically because both the variational distribution $Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x})$ and the prior $P(Z)$ are normal distributions (we assume here the $P(Z) = \mathcal N(0, I)$ is a standard normal distribution, i.e. it has zero mean and unit standard deviation).
According to Wikipedia, the [KL-divergence between two normal distributions](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions) $\mathcal N_0$ and $\mathcal N_1$ with means $\mu_0$ and $\mu_1$, respectively, and covariance matrices $\Sigma_0$ and $\Sigma_1$, respectively, is given by

$$
  D_\text{KL}\left(\mathcal{N}_0 \parallel \mathcal{N}_1\right) =
  \frac{1}{2}\left(
    \operatorname{tr}\left(\Sigma_1^{-1}\Sigma_0\right)  - k +
    \left(\mu_1 - \mu_0\right)^\mathsf{T} \Sigma_1^{-1}\left(\mu_1 - \mu_0\right) +
    \ln\left(\frac{\det\Sigma_1}{\det\Sigma_0}\right)
  \right).
$$
### Your Task

Complete the implementation of the function `kl_divergence_from_prior` (i.e., fill in the gap labeled with `TODO`) by evaluating the above expression for the KL-divergence.
You shouldn't have to explicitly construct any matrices.
Since all matrices are diagonal, you can rewrite everything as a simple term inside the `torch.sum(...)` below.
Use the following substitutions:
- mean of the variational distribution: $\mu_0 = \texttt{q\_mean}$
- mean of the prior: $\mu_1 = 0$
- covariance matrix of the variational distribution: $\Sigma_0 = \text{diag}(\texttt{q\_variance})$
- covariance matrix of the prior: $\Sigma_1 = I = \text{diag}(1,\ldots,1)$
- dimensionality of $Z$ (the "$-k$" term in the above KL-divergence): simply subtract $1$ inside `torch.sum(...)` below; this will subtract $1$ from each of the $k$ components over which we `sum`.
- $\det \Sigma_1 = \det I = 1$
- $\ln\det\Sigma_0 = \operatorname{tr}\ln\Sigma_0 = \texttt{torch.sum(q\_log\_variance)}$

In [ ]:
def kl_divergence_from_prior(q_mean, q_log_variance):
    q_variance = q_log_variance.exp()
    return 0.5 * torch.sum(TODO)

## Evidence Lower Bound (ELBO)

The following function estimates the ELBO,
$$
\text{ELBO}(\phi) = \mathbb E_{Q_\phi(Z|\mathbf{X}=\mathbf{x})}\big[ \log P(\mathbf{X}\! =\!\mathbf{x}\,|\,Z)\big] - D_\text{KL}\big(Q_\phi(Z\,|\,\mathbf{X}\! =\!\mathbf{x}) \,\big|\;\!\!\big|\,  P(Z)\big).
$$

Read the implementation and make sure you understand it.

In [ ]:
def elbo_and_reconstruction(encoder, decoder, x):
    q_mean, q_log_variance = encoder(x)
    z = reparameterize(q_mean, q_log_variance)
    x_reconstructed = decoder(z)
    log_likelihood = -F.binary_cross_entropy(x_reconstructed, x, reduction='sum')

    elbo = log_likelihood - kl_divergence_from_prior(q_mean, q_log_variance)
    return elbo, x_reconstructed

## Training and Testing

The remaining parts of this notebook train the VAE by maximizing the ELBO over both the encoder and the decoder parameters (i.e., via amortized variational expectation maximization).
Read the implementation, make sure you understand what is going on.
Then try it out.
You should observe that the ELBO increases over the course of training, and that both samples and reconstructions appear to increase in quality.

You could use the trained model to losslessly compress an MNIST image using bits-back coding, and the resulting net bit rate should be close to the negative ELBO (you'll have to quantize $z$-space so that you can losslessly compress $z$, but as long as your quantization grid is fine enough the precise form of quantization won't affect the net bit rate).

On the next problem set we will modify the VAE that we introduced here so that you use it for *lossy* compression.

In [ ]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)
if torch.cuda.device_count() > 1:
    encoder = nn.DataParallel(encoder)
    decoder = nn.DataParallel(decoder)
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)

In [ ]:
def train(epoch):
    # Set encoder and decoder models into training mode.
    encoder.train()
    decoder.train()

    total_elbo = 0
    for batch_idx, (batch, _) in enumerate(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        current_elbo, _ = elbo_and_reconstruction(encoder, decoder, batch)
        loss = -current_elbo
        loss.backward()
        total_elbo += current_elbo.item()
        optimizer.step()
        if batch_idx % BATCHES_BETWEEN_LOGGING == 0:
            print(
                f'Training epoch {epoch} [{batch_idx * len(batch)}/{len(train_loader.dataset)}' +
                f' ({100. * batch_idx / len(train_loader):.0f}%)]:' +
                f' ELBO of current batch = {current_elbo.item() / (len(batch) * np.log(2)):.6f}'
            )
            sys.stdout.flush()

    return total_elbo / len(train_loader.dataset)

In [ ]:
def test( render_reconstructions=False):
    # Set encoder and decoder models into evaluation mode.
    encoder.eval()
    decoder.eval()

    rendered_reconstructions = None
    total_elbo = 0
    with torch.no_grad():
        for i, (batch, _) in enumerate(test_loader):
            batch = batch.to(device)
            current_elbo, reconstruction = elbo_and_reconstruction(encoder, decoder, batch)
            total_elbo += current_elbo
            if i == 0 and render_reconstructions:
                n = min(batch.size(0), 8)
                comparison = torch.cat([batch[:n],
                                        reconstruction.view(BATCH_SIZE, 1, 28, 28)[:n]])
                rendered_reconstructions = make_grid(comparison.cpu()).permute((1, 2, 0))

    return total_elbo / len(test_loader.dataset), rendered_reconstructions

In [ ]:
def create_figure():
    figure, (samples_plot, reconstructions_plot) = plt.subplots(1, 2)
    samples_plot.set_title('samples')
    samples_plot.xaxis.set_visible(False)
    samples_plot.yaxis.set_visible(False)
    reconstructions_plot.set_title('test images and\ntheir reconstructions')
    reconstructions_plot.xaxis.set_visible(False)
    reconstructions_plot.yaxis.set_visible(False)
    return figure, samples_plot, reconstructions_plot

for epoch in range(1, EPOCHS + 1):
    start_time = time.time()
    show_plots = epoch == 1 or epoch % 5 == 0  

    train_elbo = train(epoch)
    test_elbo, reconstructions = test(render_reconstructions=show_plots)
    end_time = time.time()
    print(
        f'Epoch {epoch} of {EPOCHS} ({end_time - start_time:.0f} sec): ' +
        f'training set ELBO = {train_elbo/np.log(2):.4f}; ' +
        f'test set ELBO = {test_elbo/np.log(2):.4f} ' +
        '(both ELBOs in base 2)\n'
    )

    if show_plots:
        figure, samples_plot, reconstructions_plot = create_figure()
        with torch.no_grad():
            sample = torch.randn(64, LATENT_DIM).to(device)
            sample = decoder(sample).cpu()
            samples_plot.imshow(make_grid(sample.view(64, 1, 28, 28)).permute((1, 2, 0)))
        reconstructions_plot.imshow(reconstructions)
        plt.show()